In [1]:
import pandas as pd
from Bio import SeqIO
import numpy as np
import pickle
import os
import json
import seaborn as sns
path_real = "/home/chensa/gb_data/data_for_analysis/part2/"
os.chdir(path_real)

In [2]:
phastEI = {}
with open("phastConsElements30way.txt") as f:
    for q in f:
        line = q.split("\t")
        chr = line[1]
        start = int(line[2]); end = int(line[3]); score = int(line[5])
        if chr not in phastEI:
            phastEI[chr] = [[],[],[]]
        phastEI[chr][0].append(start); phastEI[chr][1].append(end); phastEI[chr][2].append(score)


In [47]:
id2all = pickle.load(open("/home/chensa/gb_data/data_for_analysis/mRNA/mRNA_id2all.pkl","rb"))
enst_list = list(id2all.keys())
df = pd.read_csv("../mRNA/Enst_Ensg_Symbol.txt", sep="\t", header=None)
enst2ensg = {}; ensg2enst = {}
for enst,ensg in zip(df[0],df[1]):
    enst2ensg[enst] = ensg
    ensg2enst[ensg] = enst
ensg_list = [enst2ensg[enst] for enst in enst_list]

df = pd.read_csv('TE2.tsv', sep="\t")
ensg_list_with_expr = df['ENSG'].tolist()

keep_ensg = list(set(ensg_list) & set(ensg_list_with_expr))
print(len(enst_list),len(ensg_list_with_expr),len(keep_ensg))

19293 28190 18904


In [67]:
ensg25utr_region = {}
ensg2strand = {}
with open("five_prime_UTR.MANE.GRCh38.v1.3.ensembl_genomic.gff") as f:
    for q in f:
        line = q.split("\t")
        ensg = line[8].split("gene_id=")[1].split(".")[0]
        start = int(line[3])
        end = int(line[4])
        chr = line[0]
        strand = line[6]
        if ensg not in keep_ensg: continue
        ensg2strand[ensg] = strand
        if ensg not in ensg25utr_region:
            ensg25utr_region[ensg] = [chr,strand]
        if start != end:
            ensg25utr_region[ensg].append((start,end))
print(len(ensg25utr_region))

18634


In [72]:
# ensg2phast = {}
ensg2phast = pickle.load(open("ensg2phast.pkl","rb"))

In [81]:
def regions_join(lst1, lst2):
    if lst2[0] >= lst1[0]:
        if lst2[0] >= lst1[1]:
            return None
        elif lst2[1] <= lst1[1]:
            return lst2
        else:
            return [lst2[0],lst1[1]]
    else:
        if lst2[1] <= lst1[0]:
            return None
        elif lst2[1] <= lst1[1]:
            return [lst1[0],lst2[1]]
        else:
            return lst1

for ensg, v in ensg25utr_region.items():
    # if ensg in ensg2phast: continue
    if ensg != 'ENSG00000071564': continue
    chr = v[0]
    strand = v[1]
    regions = sorted(v[2:], key=lambda x:x[0])
    phastStarts = phastEI[chr][0]; phastEnds = phastEI[chr][1]; phastScores = phastEI[chr][2]

    if strand == "+":
        phast = []
        shift = 0
        for start, end in regions:
            tmp_phast = []
            start_idx = np.searchsorted(phastStarts, start+0.5)
            end_idx = np.searchsorted(phastEnds, end-0.5)
            if start_idx>0:start_idx -= 1
            if end_idx==len(phastStarts):end_idx -= 1
            for idx in range(start_idx, end_idx+1):
                lst = regions_join([start,end],[phastStarts[idx],phastEnds[idx]])
                if lst is None: continue
                lst[0] -= start; lst[1] -= start
                lst[0] += shift; lst[1] += shift
                lst.append(phastScores[idx])
                tmp_phast.append(lst)
            shift += end - start + 1
        phast = [sublist for sublist in tmp_phast]
        phast = sorted(phast, key=lambda x:x[0])
        ensg2phast[ensg] = phast
    
    elif strand == "-":
        phast = []
        shift = 0
        for start, end in regions[:1]:
            tmp_phast = []
            start_idx = np.searchsorted(phastStarts, start+0.5)
            end_idx = np.searchsorted(phastEnds, end-0.5)
            if start_idx>0:start_idx -= 1
            if end_idx==len(phastStarts):end_idx -= 1
            for idx in range(start_idx, end_idx+1):
                lst = regions_join([start,end],[phastStarts[idx],phastEnds[idx]])
                # print(idx)
                # print([phastStarts[idx],phastEnds[idx]])
                # print(lst)
                if lst is None: continue
                tmp_var = lst[0]; lst[0] = end - lst[1]; lst[1] = end - tmp_var
                lst[0] += shift; lst[1] += shift
                lst.append(phastScores[idx])
                tmp_phast.append(lst)
            shift += end - start + 1
        phast = [sublist for sublist in tmp_phast]
        phast = sorted(phast, key=lambda x:x[0])
        ensg2phast[ensg] = phast

In [69]:
ensg25utr_region['ENSG00000037637'], ensg25utr_region['ENSG00000157916'],ensg25utr_region['ENSG00000157388']

(['chr1', '-', (16352255, 16352480), (16315419, 16315435)],
 ['chr1', '+', (2391841, 2391958), (2395784, 2395790)],
 ['chr3', '+', (53494611, 53495166), (53494611, 53495166)])

In [77]:
pickle.dump(ensg2phast, open("ensg2phast.pkl","wb"))

In [48]:
df = pd.read_csv("TE2.tsv", sep="\t",)
long_list = []; long_lens = []
name2expr = {}
for row in df.iterrows():
    ensg = row[1][0]
    if ensg not in keep_ensg:continue
    enst = ensg2enst[ensg]
    info = id2all[enst]
    if info['start'] < 300: continue
    tpms = np.array([float(x) for x in row[1][1:]])
    tpms = np.mean([_ for _ in tpms if _ == _])
    if tpms != tpms: continue
    name2expr[ensg] = tpms
    long_list.append(ensg);long_lens.append(info['start'])
print(len(long_list))

/home/chensa/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/chensa/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


3663


In [49]:
labels = pd.qcut(long_lens, q=50, labels=False)

bins = np.percentile(long_lens, np.linspace(0, 100, 50 + 1))
labels, bins

(array([34,  9, 31, ..., 16, 39,  3]),
 array([ 300.  ,  303.24,  308.  ,  311.  ,  316.  ,  320.  ,  325.  ,
         330.  ,  334.  ,  339.  ,  345.  ,  350.  ,  354.  ,  360.  ,
         366.  ,  373.  ,  379.84,  385.  ,  391.  ,  399.  ,  406.  ,
         413.  ,  421.  ,  428.  ,  436.  ,  444.  ,  452.  ,  459.  ,
         468.72,  478.  ,  487.  ,  500.44,  514.  ,  524.92,  539.  ,
         552.  ,  565.  ,  585.  ,  606.  ,  623.  ,  642.  ,  668.  ,
         695.  ,  728.  ,  767.  ,  809.  ,  872.  ,  959.  , 1092.52,
        1287.52, 3561.  ]))

In [50]:
bins_ensg = [[] for _ in range(50)]
for ensg, utrlen in zip(long_list, long_lens):
    idx = np.searchsorted(bins, utrlen)
    bins_ensg[idx-1].append(ensg)

In [51]:
highensg = []; lowensg = []
for i in range(50):
    ensg_bin = bins_ensg[i]
    exprs = [name2expr[ensg] for ensg in ensg_bin]
    highthr,lowthr = np.percentile(exprs, [80,20])
    highensg += [ensg for ensg in ensg_bin if name2expr[ensg] > highthr]
    lowensg += [ensg for ensg in ensg_bin if name2expr[ensg] < lowthr]
len(highensg), len(lowensg)

(745, 745)

In [52]:
len(name2expr)

3663

In [82]:
low2phast = {k:{'phast':v,'utr_len':0} for k,v in ensg2phast.items() if k in lowensg}
high2phast = {k:{'phast':v,'utr_len':0} for k,v in ensg2phast.items() if k in highensg}
low_len = {}
for k,v in low2phast.items():
    utr_len = id2all[ensg2enst[k]]['start']
    low2phast[k]['utr_len'] = utr_len
    low_len[k] = utr_len
high_len = {}
for k,v in high2phast.items():
    utr_len = id2all[ensg2enst[k]]['start']
    high2phast[k]['utr_len'] = utr_len
    high_len[k] = utr_len
# low_len按照utr长度排序得到keys顺序
sorted_keys = sorted(low_len, key=lambda x:low_len[x])
new_low2phast = []; new_low_len = []
new_low2phast_names = []
for k in sorted_keys:
    new_low2phast.append(low2phast[k])
    new_low_len.append(low_len[k])
    new_low2phast_names.append(k)
sorted_keys = sorted(high_len, key=lambda x:high_len[x])
new_high2phast = []; new_high_len = []
new_high2phast_names = []
for k in sorted_keys:
    new_high2phast.append(high2phast[k])
    new_high2phast_names.append(k)
    new_high_len.append(high_len[k])

In [83]:
high2phast['ENSG00000157388'], ensg2phast['ENSG00000071564'], id2all[ensg2enst['ENSG00000157388']],ensg25utr_region['ENSG00000157388']

({'phast': [[0, 27, 339], [54, 373, 600], [380, 555, 619]], 'utr_len': 556},
 [],
 {'rna': 'GAAGCCGGCGTGCGGCGCGGCGCGGCGGGCGCGGAGCGAGCGGGCGGGCGAGCGCCTCCGTCCCCGGATGTGAGCTCCGGCTGCCCGCGGTCCCGAGCCAGCGGCGGCGCGGGCGGCGGCGGCGGGCACCGGGCACCGCGGCGGGCGGGCAGACGGGCGGGCATGGGGGGAGCGCCGAGCGGCCCCGGCGGCCGGGCCGGCATCACCGCGGCGTCTCTCCGCTAGAGGAGGGGACAAGCCAGTTCTCCTTTGCAGCAAAAAATTACATGTATATATTATTAAGATAATATATACATTGGATTTTATTTTTTTAAAAAGTTTATTTTGCTCCATTTTTGAAAAAGAGAGAGCTTGGGTGGCGAGCGGTTTTTTTTTTAAATCAATTATCCTTATTTTCTGTTATTTGTCCCCGTCCCTCCCCACCCCCCTGCTGAAGCGAGAATAAGGGCAGGGACCGCGGCTCCTACCTCTTGGTGATCCCCTTCCCCATTCCGCCCCCGCCTCAACGCCCAGCACAGTGCCCTGCACACAGTAGTCGCTCAATAAATGTTCGTGGATGATGATGATGATGATGATGAAAAAAATGCAGCATCAACGGCAGCAGCAAGCGGACCACGCGAACGAGGCAAACTATGCAAGAGGCACCAGACTTCCTCTTTCTGGTGAAGGACCAACTTCTCAGCCGAATAGCTCCAAGCAAACTGTCCTGTCTTGGCAAGCTGCAATCGATGCTGCTAGACAGGCCAAGGCTGCCCAAACTATGAGCACCTCTGCACCCCCACCTGTAGGATCTCTCTCCCAAAGAAAACGTCAGCAATACGCCAAGAGCAAAAAACAGGGTAACTCGTCCAACAGCCGACCTGCCCGCGCCCTTTTCTGTTTATCACTCAATAACCCCATCCGAAGAG

In [90]:
# 画矩形块(2476289, 2476588)，(2476289, 2476225)的颜色深度261，(2476630, 2476588)的颜色深度300，其余白色
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches
high_plot_regions = [(0,350), (350,650), (650, len(new_high2phast))]
low_plot_regions = [(0,350), (350,650), (650, len(new_low2phast))]
max_lens = [431,755,2540]
cmap = plt.get_cmap('PuBu')
norm = plt.Normalize(vmin=0,vmax=1000)
high2block = {}; low2block = {}
for p in range(3):
    high_start, high_end = high_plot_regions[p]; low_start, low_end = low_plot_regions[p]; max_len = max_lens[p]
    fig = plt.figure(figsize=(6,int(high_end-high_start)/50))
    titles = ["high TE", "low TE"]
    for idx,data in enumerate([new_high2phast[high_start:high_end], new_low2phast[low_start:low_end]]):
     
        end_len_all = []
        names = new_high2phast_names[high_start:high_end] if idx == 0 else new_low2phast_names[low_start:low_end]

        ax = fig.add_subplot(1,2,idx+1)
        num = len(data); width = 1 / num; pidx = 0

        for name,v in zip(names,data):
            r_id = 0
            block = 0
            utr_len = v['utr_len']
            tmp_block = [utr_len]
            if v['phast'] == []:
                ax.add_patch(patches.Rectangle((0, pidx*width), 1000, width, color='white'))
                pidx += 1
                if idx == 0:
                    high2block[name] = tmp_block
                else:
                    low2block[name] = tmp_block
                continue
            phast = []
            for phast in v['phast']:
                flag = False
                start = phast[0]; end = phast[1]; score = phast[2]
                # if score > 500 and end - start > 100:
                if True:
                    color = (1-score/1000,0,0)
                    # color = cmap(norm(score)) 
                    tmp_block.append((score,end-start))
                    ax.add_patch(patches.Rectangle((start , pidx*width), end-start, width, color=color))
                    flag = True
                    if end > utr_len:
                        print(name,utr_len,end)
                
            ax.add_patch(patches.Rectangle((utr_len , pidx*width-width/2), 1, width*2, color='grey'))
            if idx == 0:
                high2block[name] = tmp_block
            else:
                low2block[name] = tmp_block
            pidx += 1
            if pidx == num: break
            end_len_all.append(utr_len)
        end_len = np.quantile(end_len_all, 1)
        ax.set_xlim(0,max_len)



        print(end_len)
        ax.set_yticks([])
        # ax.set_xlabel("low 5'UTR length")
        if p != 1:
            ax.set_xlabel(titles[idx],fontsize=16)
    # break
    plt.savefig(f"figures/phast_high_low_{p}.pdf", bbox_inches='tight', format='pdf')
    # plt.savefig(f"figures/phast_high_low_{p}_500.pdf", bbox_inches='tight', format='pdf')
    plt.close()

429
431
743
755
1676
2540


In [87]:
print(len(high2block), len(low2block))
high_utr_lens = []; low_utr_lens = []
highscores = []; lowscores = []
for k,v in high2block.items():
    high_utr_lens.append(v[0])
    if len(v) > 1:
        utr_len = v[0]*1000; tmp_score = 1
        for score,l in v[1:]:
            tmp_score += score*l
        highscores.append(tmp_score/utr_len)
    else:
        highscores.append(0)
for k,v in low2block.items():
    low_utr_lens.append(v[0])
    if len(v) > 1:
        utr_len = v[0]*1000; tmp_score = 1
        for score,l in v[1:]:
            tmp_score += score*l
        lowscores.append(tmp_score/utr_len)
    else:
        lowscores.append(0)
from scipy.stats import wilcoxon,ttest_ind
ttest_ind(highscores, lowscores),wilcoxon(highscores, lowscores)

745 745


(TtestResult(statistic=2.77854950080061, pvalue=0.005528840916388316, df=1488.0),
 WilcoxonResult(statistic=15345.0, pvalue=0.004526059816951237))

In [88]:
'''
cut500
(TtestResult(statistic=2.77854950080061, pvalue=0.005528840916388316, df=1488.0),
 WilcoxonResult(statistic=15345.0, pvalue=0.004526059816951237))

cut0
(TtestResult(statistic=4.460445953458336, pvalue=8.793392683052747e-06, df=1488.0),
 WilcoxonResult(statistic=99042.0, pvalue=3.8404510132139645e-07))

单端检验以上/2
'''
0.0045260/2,3.8404510132139645e-07/2

(0.002263, 1.9202255066069823e-07)

In [46]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

cmap = LinearSegmentedColormap.from_list('red_black', [(1, 0, 0), (0, 0, 0)], N=256)

fig, ax = plt.subplots(figsize=(15, 1))

gradient = np.linspace(0, 1, 100)
gradient = np.vstack((gradient, gradient))

ax.imshow(gradient, aspect='auto', cmap=cmap, origin='lower')

cbar = plt.colorbar(ax.imshow(gradient, aspect='auto', cmap=cmap, origin='lower'),
                    ax=ax, orientation='horizontal')
cbar.set_label('Phast Score',fontsize=50)
cbar.set_ticks([0, 1])
cbar.set_ticklabels(['0', '1000'],fontsize=50)

ax.set_xticks([])
ax.set_yticks([])

plt.savefig("figures/colorbar.pdf", bbox_inches='tight', format='pdf')
plt.close()